<a href="https://colab.research.google.com/github/HarshCasper/Brihaspati/blob/master/Chatbot%20using%20LSTM/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle
from numpy import loadtxt

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

In [0]:
from google.colab import files
uploaded = files.upload()

Saving sample_conversations.csv to sample_conversations (1).csv


In [0]:
# load dataset
a = loadtxt('sample_conversations.csv', delimiter=',', dtype=str)

# reduce dataset size
n_sentences = 29
dataset = a[:n_sentences, :]

# random shuffle
shuffle(dataset)

# split into train/test
train, test = dataset[:29], dataset[29:]

# save
save_clean_data(dataset, 'both.pkl')
save_clean_data(train, 'train.pkl')
save_clean_data(test, 'test.pkl')

Saved: both.pkl
Saved: train.pkl
Saved: test.pkl


In [0]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# intereply encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

# define NMT model
def define_model(vocab, timesteps, n_units):
	model = Sequential()
	model.add(Embedding(vocab, n_units, input_length=timesteps, mask_zero=True))
	model.add(LSTM(n_units))
	model.add(RepeatVector(timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(TimeDistributed(Dense(vocab, activation='softmax')))
	return model

# load datasets
dataset = load_clean_sentences('both.pkl')
dataset1 = dataset.reshape(-1,1)
train = load_clean_sentences('train.pkl')
test = load_clean_sentences('test.pkl')

# prepare tokenizer
all_tokenizer = create_tokenizer(dataset1[:,0])
all_vocab_size = len(all_tokenizer.word_index) + 1
all_length = max_length(dataset1[:, 0])
print('ALL Vocabulary Size: %d' % (all_vocab_size))
print('ALL Max question length: %d' % (all_length))

# prepare training data
trainX = encode_sequences(all_tokenizer, all_length, train[:, 0])
trainY = encode_sequences(all_tokenizer, all_length, train[:, 1])
trainY = encode_output(trainY, all_vocab_size)

# prepare validation data
testX = encode_sequences(all_tokenizer, all_length, test[:, 0])
testY = encode_sequences(all_tokenizer, all_length, test[:, 1])
testY = encode_output(testY, all_vocab_size)

# define model
model = define_model(all_vocab_size, all_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')

# summarize defined model
print(model.summary())

#train and save model
model.fit(trainX, trainY, epochs=500, batch_size=64, verbose=1)
filename = 'model1.h5'
model.save(filename)

ALL Vocabulary Size: 56
ALL Max question length: 10
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 256)           14336     
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 10, 256)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 256)           525312    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 56)            14392     
Total params: 1,079,352
Trainable params: 1,079,352
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/500
29/29 [==============================] - 1s 27ms/step - loss: 4.0254
Epoch 2/500
29/29 [==============================] - 0s 4ms/step - loss: 3.9917
Epoch 3/500
29/29 [==============================] - 0s 4ms/step - loss: 3.9470
Epoch 4/500
29/29 [==============================] - 0s 4ms/step - loss: 3.8753
Epoch 5/500
29/29 [==============================] - 0s 4ms/step - loss: 3.7558
Epoch 6/500
29/29 [==============================] - 0s 4ms/step - loss: 3.5537
Epoch 7/500
29/29 [==============================] - 0s 4ms/step - loss: 3.2148
Epoch 8/500
29/29 [==============================] - 0s 4ms/step - loss: 2.7155
Epoch 9/500
29/29 [==============================] - 0s 4ms/step - loss: 2.2483
Epoch 10/500
29/29 [==============================] - 0s 4ms/step - loss: 1.9930
Epoch 11/500
29/29 [==============================] - 0s 4ms/step - loss: 1.9495
Epoch 12/500
29/29 [==============================] - 0s 4ms/step - loss: 2.0065
Epoch 13/500
29/29 [================

In [0]:
from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer(char_level=False)
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

# translate
def translate(model, tokenizer, sources):
	predicted = list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, all_tokenizer, source)
		print('ANSWER: %s' % (translation))
		predicted.append(translation.split())

# load datasets
dataset = load_clean_sentences('both.pkl')
dataset1=dataset.reshape(-1,1)

# prepare tokenizer
all_tokenizer = create_tokenizer(dataset1[:,0])
all_vocab_size = len(all_tokenizer.word_index) + 1
all_length = max_length(dataset1[:, 0])

# load model
model = load_model('model1.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
while(True):
    q = (input(str("YOU: ")))
    if q == 'bye':
        break
    q = q.strip().split('\n')

    #we tokenize
    X = all_tokenizer.texts_to_sequences(q)
    X = pad_sequences(X, maxlen=all_length, padding='post')
        
    # find reply and print it out
    translate(model, all_tokenizer, X)

YOU: Hi
ANSWER: hello
YOU: How are you?
ANSWER: good and you
